# ART for scikit-learn LogisticRegression

In [ ]:
from sklearn.svm import SVC

import numpy as np
from matplotlib import pyplot as plt

from art.classifiers import SklearnSVC
from art.attacks.projected_gradient_descent import ProjectedGradientDescent
from art.utils import load_mnist

# Load and transform MNIST dataset

In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_mnist()

n_samples_train = x_train.shape[0]
n_features_train = x_train.shape[1] * x_train.shape[2] * x_train.shape[3]
n_samples_test = x_test.shape[0]
n_features_test = x_test.shape[1] * x_test.shape[2] * x_test.shape[3]

x_train = x_train.reshape(n_samples_train, n_features_train)
x_test = x_test.reshape(n_samples_test, n_features_test)

y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

n_samples_max = 100
x_train = x_train[0:n_samples_max]
y_train = y_train[0:n_samples_max]
x_test = x_test[0:n_samples_max]
y_test = y_test[0:n_samples_max]

# Train LogisticRegression classifier

In [ ]:
skl_svc = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto_deprecated', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

In [ ]:
skl_svc.fit(X=x_train, y=y_train)

# Create and apply ProjectedGradientDescent Attack with ART

In [ ]:
art_svc = SklearnSVC(model=skl_svc)

In [ ]:
pgd = ProjectedGradientDescent(classifier=art_svc, norm=np.inf, eps=.3, eps_step=0.1, max_iter=20, targeted=False, num_random_init=0,
                 batch_size=128)

In [ ]:
x_train_adv = pgd.generate(x_train)

In [ ]:
x_test_adv = pgd.generate(x_test)

# Evaluate LogisticRegression classifier on benign and adversarial samples

In [ ]:
score = skl_svc.score(x_train, y_train)
print("Benign Training Score: %.4f" % score)

In [ ]:
plt.matshow(x_train[0, :].reshape((28, 28)))
plt.clim(0, 1)

In [ ]:
prediction = skl_svc.predict(x_train[0:1, :])[0]
print("Benign Training Predicted Label: %i" % prediction)

In [ ]:
score = skl_svc.score(x_train_adv, y_train)
print("Adversarial Training Score: %.4f" % score)

In [ ]:
plt.matshow(x_train_adv[0, :].reshape((28, 28)))
plt.clim(0, 1)

In [ ]:
prediction = skl_svc.predict(x_train_adv[0:1, :])[0]
print("Adversarial Training Predicted Label: %i" % prediction)

In [ ]:
score = skl_svc.score(x_test, y_test)
print("Benign Test Score: %.4f" % score)

In [ ]:
plt.matshow(x_test[0, :].reshape((28, 28)))
plt.clim(0, 1)

In [ ]:
prediction = skl_svc.predict(x_test[0:1, :])[0]
print("Benign Test Predicted Label: %i" % prediction)

In [ ]:
score = skl_svc.score(x_test_adv, y_test)
print("Adversarial Test Score: %.4f" % score)

In [ ]:
plt.matshow(x_test_adv[0, :].reshape((28, 28)))
plt.clim(0, 1)

In [ ]:
prediction = skl_svc.predict(x_test_adv[0:1, :])[0]
print("Adversarial Test Predicted Label: %i" % prediction)

# Investigate dependence on attack budget eps

In [ ]:
eps_list = [0.1, 0.2, 0.3, 0.4, 0.5]
score_list = list()

for eps in eps_list:
    pgd = ProjectedGradientDescent(classifier=art_svc, norm=np.inf, eps=eps, eps_step=0.05, max_iter=20, targeted=False, num_random_init=0,
                 batch_size=128)
    x_test_adv = pgd.generate(x_test)
    score = skl_svc.score(x_test_adv, y_test)
    score_list.append(score)

plt.plot(eps_list, score_list)
plt.xlabel('eps')
plt.ylabel('Test Accuracy')
plt.ylim((0, 1))

# Targeted PGD attack

In [ ]:
pgd = ProjectedGradientDescent(classifier=art_svc, norm=np.inf, eps=0.5, eps_step=0.01, max_iter=500, targeted=True, num_random_init=3, batch_size=128)
y_test_target = np.zeros((y_test.shape[0], 10))
target_label = 7
y_test_target[:, target_label] = 1
x_test_adv = pgd.generate(x_test, y=y_test_target)
score = skl_svc.score(x_test_adv, y_test)

In [ ]:
score = skl_svc.score(x_test_adv, np.argmax(y_test_target, axis=1))
print("Targeted Benign Test Score: %.4f" % score)

In [ ]:
plt.matshow(x_test_adv[16, :].reshape((28, 28)))
plt.clim(0, 1)

In [ ]:
prediction = skl_svc.predict(x_test_adv[16:17, :])[0]
print("Targeted Benign Test Predicted Label: %i" % prediction)